In [6]:
# Dominik Csatári | FV1TW4 | 2023.10.13.
# My Kaggle User is: Dominik Csatári | https://www.kaggle.com/dominikcsatri
# For the codes, I've created are highly contributed wiht general documentation code
# from https://scikit-learn.org  as this is where I have learned, how to use, each
# type of estimator, or what hyperparemeters can be tuned, what libraries are recommended to use.

#Set random_seed:
random_seed = 42

In [2]:
#imports
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import csv
from sklearn.ensemble import RandomForestRegressor
from tabulate import tabulate


In [3]:
# At this part i read in the files.
X = pd.read_csv('pc_X_train.csv')
Final_X_test = pd.read_csv('pc_X_test.csv')
Y = pd.read_csv('pc_y_train.csv')


In [4]:
# First drop id attribute, as it is has no use, regarding predicitions.
X = X.drop(X.columns[0], axis=1)
Final_x_test = Final_X_test.drop(Final_X_test.columns[0], axis=1)

In [5]:
# Split oour data for training and testing our model
random_seed = 42
X_train, X_test, Y_train, Y_test = train_test_split(X, Y['score'], test_size=0.2, random_state=random_seed)


In [6]:
# This function is for writing out our final predicted y to a file for later upload
def WriteOutput(data,file_path="output.csv"):
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        writer.writerow(['id', 'score'])
        
        for idx, score in enumerate(data):
            writer.writerow([idx, score])



RandomForest

In [46]:
#Random forest 
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model on the training set
rf_model.fit(X_train, Y_train)

# Make predictions on the testing set
y_pred = rf_model.predict(X_test)



In [47]:
# This is the mse result for random forest:
# (With the feutre reduction, with has not tbeen fine-tuned yet, meaning
# it is mainly the same result that we would have got with simple X)
mse_varSelected = mean_squared_error( Y_test, y_pred)
mse_varSelected

0.4046240530303031

In [32]:
# Finla prediciton for kaggle upload
Final_pred = rf_model.predict(Final_X_test)


In [33]:
WriteOutput(Final_pred,'output01.csv')

Try Entropy:

In [42]:
# Ill try Absolute error so outliers less likely to miss
rf_model = RandomForestRegressor(n_estimators=150, criterion="absolute_error",random_state=42)

# Train the model on the training set
rf_model.fit(X_train, Y_train)

# Make predictions on the testing set
y_pred = rf_model.predict(X_test)

mse_rf = mean_squared_error( Y_test, y_pred)
mse_rf
#Its a bit worse, with mse it was 0.39

0.41764873737373737

Feature reduction

In [36]:
# https://scikit-learn.org/stable/modules/feature_selection.html,
# This part ha not been fine-tuned yet, no actual change has been made by this block.
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
x_new = sel.fit_transform(X)

#For this dataset we get 0.4 error with the first setup of randomforest
X_train, X_test, Y_train, Y_test = train_test_split(x_new, Y['score'], test_size=0.2, random_state=random_seed)

XBoost

In [5]:
# Resolved an error with this transofrmation help:
# https://stackoverflow.com/questions/71996617/invalid-classes-inferred-from-unique-values-of-y-expected-0-1-2-3-4-5-got
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
Y_train = le.fit_transform(Y_train)


In [28]:
from xgboost import XGBRegressor

learn = XGBRegressor(n_estimators = 300, learning_rate = 0.001, max_depth = 10)
learn.fit(X_train, Y_train)

y_pred = learn.predict(X_test)
mse_xb = mean_squared_error( Y_test, y_pred)



In [12]:
#n_estimators = 200, learning_rate = 0.01, max_depth = 10, this was the result I have uploaded initially.
mse_xb = mean_squared_error( Y_test, y_pred)
mse_xb

0.5142629966858754

In [29]:
# XGBRegressor(n_estimators = 300, learning_rate = 0.001, max_depth = 10)
mse_xb = mean_squared_error( Y_test, y_pred)
mse_xb

0.7289235133649757

In [22]:
Final_y_pred = learn.predict(Final_x_test)

In [27]:
WriteOutput(Final_y_pred,"out_XBoost.csv")

SVM

In [47]:
# For this part I've applied a GridSearch for HyperParameter Tuning method, Exhaustive GridSearch.
# I've searched for the best model in Models, I've saved, given paramteres from a predefined matrix, Models_res.
from sklearn import svm

reg_numb = 30;
eps_values = 20;

regularization = np.linspace(0.008,1,reg_numb)
epsilon_value = np.linspace(0.1,0.8,eps_values)

Models_res = np.zeros((reg_numb, eps_values))
Models = []

for i in range(len(regularization)):
    for j in range(len(epsilon_value)):
        regr = svm.SVR(C=regularization[i],epsilon=epsilon_value[j])

        Models.append(regr)
        regr.fit(X_train, Y_train)
        y_pred=regr.predict(X_test)
        mse_SVM = mean_squared_error( Y_test, y_pred)
        Models_res[i][j] = mse_SVM



In [48]:
# ChatGBT help
min_index = np.unravel_index(np.argmin(Models_res, axis=None), Models_res.shape)
print(min_index)
print(Models_res[min_index])
print(tabulate(Models_res))

(2, 13)
0.7240797743811791
--------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------
0.756711  0.755161  0.753307  0.750511  0.749576  0.752871  0.755288  0.757292  0.759656  0.762463  0.766102  0.769252  0.768278  0.76491   0.761917  0.758878  0.75748   0.756395  0.755889  0.756794
0.729317  0.729319  0.729735  0.73042   0.729904  0.730401  0.729089  0.728432  0.72944   0.731585  0.730652  0.730899  0.731349  0.729867  0.72849   0.727532  0.728491  0.730545  0.731337  0.729472
0.728552  0.729663  0.730825  0.731752  0.732383  0.731678  0.729715  0.727389  0.726691  0.725186  0.725233  0.725402  0.725194  0.72408   0.724989  0.726112  0.72787   0.730393  0.73075   0.728791
0.728893  0.729589  0.730969  0.732099  0.732623  0.732403  0.730158  0.728402  0.726747  0.725443  0.72503   0.724394  0.724524  0.724712  0.725104  0.726159  0.728258  0.73057

In [49]:
# (x,y) index to linear index:
ind = (min_index[0]-1)*reg_numb+min_index[1]
Best_SVM_Model = Models[ind]

In [66]:
Final_SVM_Pred = Best_SVM_Model.predict(Final_x_test)
WriteOutput(Final_SVM_Pred,"SVM_Output.csv")

KNN


In [82]:
# For KNN, It was hard to define a good initial estimate of the correct parameters, so I've turned to other methods.
from sklearn.neighbors import KNeighborsRegressor

neigh = KNeighborsRegressor(n_neighbors=70,weights='distance')
neigh.fit(X_train, Y_train)
y_pred = neigh.predict(X_test)
mse_KNN = mean_squared_error( Y_test, y_pred)
rmse_KNN = np.sqrt(mse_KNN)

print(rmse_KNN)

0.8673441517785204


In [83]:
Final_KNN_Pred = neigh.predict(Final_x_test)
WriteOutput(Final_KNN_Pred,"KNN_Output.csv")

AdaBoost

In [89]:
from sklearn.ensemble import AdaBoostRegressor


0.6480723623018834


In [ ]:
# For this part I've applied a GridSearch for HyperParameter Tuning method, Exhaustive GridSearch.
# I've searched for the best model in Models, I've saved, given paramteres from a predefined matrix, Models_res.
lr_numb = 30;
numb_trees = 10;

learning_rate = np.linspace(0.001,1,lr_numb)
trees = np.linspace(70,120,numb_trees)

Models_res = np.zeros((numb_trees, lr_numb))
Models = []

for i in range(len(trees)):
    for j in range(len(learning_rate)):
                
        AdaBoost = AdaBoostRegressor(random_state=42, n_estimators=int(trees[i]), learning_rate = learning_rate[j])
        Models.append(AdaBoost)

        AdaBoost.fit(X_train, Y_train)

        y_pred = AdaBoost.predict(X_test)
        mse_ada = mean_squared_error( Y_test, y_pred)
        rmse_ada = np.sqrt(mse_ada)

        Models_res[i][j] = rmse_ada
        


In [106]:
# For min_index we get the right value for the best estiamter.
# (I had stop the runing, hence the 0's, but when searching for the best estimator, I've disregarded them. )
min_index = np.unravel_index(np.argmin(Models_res, axis=None), Models_res.shape)

print(min_index)
print(Models_res[min_index])
print(tabulate(Models_res))
ind = (min_index[0]-1)*lr_numb+min_index[1]

Best_AdaBoost_Model = Models[ind]
print(Models[ind])

(7, 15)
0.6447542631134845
--------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------  --------
0.678024  0.667399  0.665222  0.659798  0.656282  0.655729  0.653166  0.657668  0.657633  0.659108  0.654097  0.658257  0.651994  0.662349  0.659229  0.649582  0.659386  0.659301  0.655524  0.661537  0.654457  0.648763  0.66222   0.65282   0.654428  0.660382  0.656543  0.652657  0.662783  0.659899
0.67635   0.6665    0.664084  0.657608  0.655046  0.653636  0.652331  0.658394  0.653441  0.658499  0.653998  0.656333  0.648936  0.65962   0.658396  0.647992  0.658785  0.660215  0.650101  0.659392  0.653252  0.650522  0.657062  0.651235  0.655224  0.659434  0.654062  0.64746   0.665761  0.654189
0.676989  0.665987  0.662897  0.658089  0.655068  0.653939  0.653206  0.6585

In [ ]:
Final_y = Best_AdaBoost_Model.predict(Final_x_test)

WriteOutput(Final_y,"AdaBoost_output.csv")

RandomForest Regressor with Hyperparameter tuning

In [1]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

In [7]:
param_grid = {"max_depth": [10, None], "min_samples_split": [5, 15]}

rf_model = RandomForestRegressor(random_state=42)

rf_model.fit(X_train, Y_train)

y_pred = rf_model.predict(X_test)

search = HalvingGridSearchCV(rf_model, param_grid, resource='n_estimators',
                             max_resources=120,
                              random_state=0).fit(X_train, Y_train)

In [9]:
search.best_params_

{'max_depth': None, 'min_samples_split': 15, 'n_estimators': 120}

In [11]:
# This gives us a values which is below the one, we have managed to gather with simple trial-and-error.
# The search cut of too early.
y_pred= search.best_estimator_.predict(X_test)
np.sqrt(mean_squared_error(y_pred,Y_test))


0.6289397996817372